In [6]:
import pandas as pd
import requests
import xml.etree.ElementTree as ET
from pathlib import Path

apiKey = "rgsaY4U.oZRQKUHdJhF9qguHMkwCGIoLaqEcaHjYLF"

def bundestag_request(resourcetype, id):
    method = "get"
    url = f'https://search.dip.bundestag.de/api/v1/{resourcetype}'
    if id is not None:
        url += "/"+str(id)
    headers = {
        'Content-type': 'application/xml',
        'Authorization': f'ApiKey {apiKey}',
    }
    rsp = requests.request(method, url, headers=headers, auth=None)
    return rsp.json()

def bundestag_xml(resourcetype, id):
    file = Path(f'../0_datasets/bundestag/{resourcetype}_{id}.xml')
    if not file.exists(): #fetch file from bundestag api if not already downloaded
        try:
            xmlurl = bundestag_request(resourcetype, id)['fundstelle']['xml_url']
        except:
            print("No xml for id " + str(id))
            return None
        rsp = requests.request("get", xmlurl)
        with open(file, 'wb') as f:
            f.write(rsp.content)

    tree = ET.parse(file)
    root = tree.getroot()
    return root

def bundestag_search(resourcetype, wahlperiode, datestart = "1900-01-01", dateend = "2900-01-01", cursor = None):
    print(cursor)
    method = "get"
    url = f'https://search.dip.bundestag.de/api/v1/{resourcetype}?f.zuordnung=BT&f.datum.start={datestart}&f.datum.end={dateend}&f.wahlperiode={wahlperiode}'
    if(cursor is not None):
        url = url + f'&cursor={cursor}'

    headers = {
        'Content-type': 'application/xml',
        'Authorization': f'ApiKey {apiKey}',
    }
    rsp = requests.request(method, url, headers=headers, auth=None)
    data = rsp.json()

    ids=[]
    for p_id in rsp.json()["documents"]:
        ids.append((p_id["id"], p_id["datum"]))

    print(ids)
    if len(ids) > 0:
        ids = ids + bundestag_search(resourcetype, wahlperiode, datestart, dateend, rsp.json()["cursor"])

    return ids

ids = bundestag_search("plenarprotokoll","19")
print(len(ids))
for id in ids:
    bundestag_xml("plenarprotokoll", id[0])

None
[('5428', '2021-09-07'), ('5427', '2021-08-25'), ('5425', '2021-06-25'), ('5424', '2021-06-24'), ('5423', '2021-06-23'), ('5421', '2021-06-11'), ('5420', '2021-06-10'), ('5419', '2021-06-09'), ('5417', '2021-05-21'), ('5416', '2021-05-20'), ('5415', '2021-05-19'), ('5413', '2021-05-07'), ('5412', '2021-05-06'), ('5411', '2021-05-05'), ('5409', '2021-04-23'), ('5408', '2021-04-22'), ('5407', '2021-04-21'), ('5406', '2021-04-16'), ('5405', '2021-04-15'), ('5404', '2021-04-14'), ('5402', '2021-03-26'), ('5401', '2021-03-25'), ('5400', '2021-03-24'), ('5398', '2021-03-05'), ('5397', '2021-03-04'), ('5396', '2021-03-03'), ('5395', '2021-02-26'), ('5394', '2021-02-25'), ('5393', '2021-02-24'), ('5391', '2021-02-12'), ('5390', '2021-02-11'), ('5389', '2021-02-10'), ('5388', '2021-01-29'), ('5387', '2021-01-28'), ('5386', '2021-01-27'), ('5384', '2021-01-14'), ('5383', '2021-01-13'), ('5380', '2020-12-17'), ('5379', '2020-12-16'), ('5378', '2020-12-11'), ('5377', '2020-12-10'), ('5376', '